In [1]:
import pandas as pd

In [24]:
df = pd.read_csv('~/Desktop/CPIAUCSL.csv', parse_dates=['observation_date'])


df.set_index('observation_date', inplace=True)


yearly_df = df.resample('A').last() 


# (CPI_current_year / CPI_previous_year - 1) * 100
yearly_df['inflation_rate'] = yearly_df['CPIAUCSL'].pct_change() * 100


yearly_df[70:78]



,CPIAUCSL,inflation_rate
observation_date,,
2017-12-31,247.805,2.129931
2018-12-31,252.767,2.002381
2019-12-31,258.630,2.319527
2020-12-31,262.005,1.304953
2021-12-31,280.808,7.176581
2022-12-31,298.812,6.411498
2023-12-31,308.742,3.323160
2024-12-31,317.685,2.896593
